In [ ]:
import os
import tensorflow as tf
import cv2
import shutil
import splitfolders
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from keras import applications
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
import seaborn as sn

In [ ]:
#import shutil

#shutil.rmtree('./tumor-datasplited')

In [ ]:
#input_folder = "./tumor-dataset"

#splitfolders.ratio(input_folder, output="tumor-datasplited", seed=42, ratio=(.7,.2,.1), group_prefix=None)


In [ ]:
try:
    #shutil.rmtree('./tumor-datasplited/.ipynb_checkpoints')
    shutil.rmtree('./tumor-datasplited/test/.ipynb_checkpoints')
    shutil.rmtree('./tumor-datasplited/train/.ipynb_checkpoints')
    shutil.rmtree('./tumor-datasplited/val/.ipynb_checkpoints')
except:
    pass

In [ ]:
train_data = ImageDataGenerator(
        rescale = 1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest',)
traindata = train_data.flow_from_directory(directory="./tumor-datasplited/train",target_size=(224,224), class_mode='categorical')

val_data = ImageDataGenerator(
        rescale = 1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest',)
valdata = val_data.flow_from_directory(directory="./tumor-datasplited/val",target_size=(224,224), class_mode='categorical')

test_data = ImageDataGenerator(
        rescale = 1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest',)
testdata = test_data.flow_from_directory(directory="./tumor-datasplited/test",target_size=(224,224), class_mode='categorical', shuffle=False)

In [ ]:
inception_model = applications.inception_v3.InceptionV3(weights='imagenet', 
                                include_top=False, 
                                input_shape=(224, 224,3))
for layers in inception_model.layers:
    layers.trainable=True

In [ ]:
inception_x = Sequential()
inception_x.add(inception_model)
inception_x.add(GlobalAveragePooling2D())
inception_x.add(Dropout(0.5))
inception_x.add(Dense(3, activation='softmax'))

In [ ]:
model = inception_x
model.compile(loss='categorical_crossentropy', optimizer = Adam(learning_rate=0.001),metrics=['accuracy'])
model.summary()

In [ ]:
checkpoint = ModelCheckpoint("INCEPTION_V3_2.h5", monitor='val_accuracy', 
                             verbose=1, save_best_only=True, 
                             save_weights_only=False, mode='auto', period=1)

early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')



In [ ]:
batch_size = 32

hist = model.fit_generator(steps_per_epoch=(traindata.n//batch_size),generator=traindata, validation_data= valdata, validation_steps=(valdata.n//batch_size),epochs=100,callbacks=[checkpoint, early])

In [ ]:
import matplotlib.pyplot as plt
acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
loss = hist.history['loss']
val_loss = hist.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, label='Acuracia de treinamento')
plt.plot(epochs, val_acc, label='Acuracia de validacao')
plt.title('Acuracia de treinamento e validacao')
plt.legend()
plt.figure()

plt.plot(epochs, loss, label='Perda de treinamento')
plt.plot(epochs, val_loss, label='Perda de validacao')
plt.title('Perda de validacao e treinamento')
plt.legend()

Y_pred = model.predict(testdata)
y_pred = np.argmax(Y_pred, axis=1)
cm = confusion_matrix(testdata.classes, y_pred)
df_cm = pd.DataFrame(cm, index = [i for i in ['glioma', 'meningioma', 'pituitary']],
                  columns = [i for i in ['glioma', 'meningioma', 'pituitary']])
plt.figure(figsize = (10,7))

sn.heatmap(df_cm, annot=True, cmap="Reds")
print('Classification Report')
target_names = ['glioma', 'meningioma', 'pituitary']
print(classification_report(testdata.classes, y_pred, target_names=target_names))

plt.show()

In [ ]:
from tensorflow import keras
model = keras.models.load_model('./INCEPTION_V3_2.h5')

In [ ]:
import matplotlib.pyplot as plt

Y_pred = model.predict(testdata)
y_pred = np.argmax(Y_pred, axis=1)
cm = confusion_matrix(testdata.classes, y_pred)
df_cm = pd.DataFrame(cm, index = [i for i in ['glioma', 'meningioma', 'pituitary']],
                  columns = [i for i in ['glioma', 'meningioma', 'pituitary']])
plt.figure(figsize = (10,7))

print(y_pred)

sn.heatmap(df_cm, annot=True, cmap="Reds")
print('Classification Report')
target_names = ['glioma', 'meningioma', 'pituitary']
print(classification_report(testdata.classes, y_pred, target_names=target_names))

plt.show()